In [87]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import scipy.optimize as opti
import wrds
import datetime
import statsmodels.api as sm

# Exercise 2

In [18]:
db = wrds.Connection() 
#goqhuB-1hafqe-dojvix

Enter your WRDS username [max]:antb95
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [19]:
names = db.get_table(library='crsp', table='stocknames')
names =names.loc[:,['permco' , 'ticker']]

In [20]:
Ticker = 'MSFT'
inter = names['permco'].where(names['ticker'] == Ticker)
inter.dropna(inplace = True)
Permco = inter.iloc[0]

In [42]:
df = db.raw_sql("select prc, date from crsp.dsf where permco in ("+str(Permco)+") and date >='2011-04-07' and date <='2016-04-07'")
df.columns = ['MSFT' , 'Date']
df.set_index('Date',inplace = True)

In [65]:
train = df.loc[df.index > datetime.date(2013,4,7)]
train = np.log(train) - np.log(train).shift(1)
train.dropna(inplace = True)
train.columns = ['R']
train['X'] = train.R - train.R.mean()
sig_0 = train.X.std()
train['Z'] = np.random.normal(0,1,len(train))

In [91]:
def Quasi_Max_Lik(coef):
    X = train.X
    Z = train.Z
    sig_0
    var = [sig_0 **2]*len(X)
    for i in range(len(X)-1):
        Y = [1 , X[i]**2 , var[i]]
        var[i+1] = np.matmul(Y,coef)
    Lik = np.sum(-np.log(var) - X**2/var)
    return [var,Lik]
        
    

In [97]:
coef = [0.2 , 0.3 , 0.5]
opti.minimize(Quasi_Max_Lik,coef)

TypeError: unsupported operand type(s) for -: 'list' and 'list'

In [68]:
alpha

0